In [1]:
import os
import openai
import json
from tqdm.auto import tqdm
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
import numpy as np

c:\Users\gooog\anaconda3\envs\nlp_clean\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
openai.api_key = os.environ.get("OPENAI_API_KEY")
openai.api_base = os.environ.get("OPENAI_API_BASE")

In [3]:
lang = "de"  # "en" or "de"

In [4]:
with open(f"instructions_{lang}.json", "r") as f:
    instructions = json.load(f)

In [8]:
stories = []
for instr in tqdm(instructions):
    theme = instr["theme"]
    character = instr["character"]
    summary = instr["summary"]
    if lang == "en":
        prompt = f"""Write a short story for children aged 3-4.
        The theme of the story is {theme}.
        The story should contain the following character: {character}
        Furthermore, the story should contain the following plot: {summary}"""
    else:
        prompt = f"""Schreibe eine kurze Geschichte für Kinder im Alter von 3-4 Jahren.
        Das Thema der Geschichte ist {theme}.
        Die Geschichte sollte den folgenden Charakter enthalten: {character}
        Außerdem sollte die Handlung der Geschichte das Folgende beinhalten: {summary}"""
    for i in range(10):
        try:
            story = openai.ChatCompletion.create(
                model = "gpt-3.5-turbo",
                stream=False,
                messages = [{"role": "user", "content": prompt}]
            ).choices[0].message.content
            break
        except Exception as e:
            story = f"AN ERROR OCCURRED! {e}"
    instr["story"] = story
    stories.append(instr)
with open(f"full_stories_{lang}.json", "w") as f:
    json.dump(stories, f, indent="    ", ensure_ascii=False)

100%|██████████| 100/100 [16:28<00:00,  9.89s/it]


In [9]:
with open(f"full_stories_{lang}.json", "r") as f:
    full_stories = json.load(f)
for fs in full_stories:
    if "ERROR" in fs["story"]:
        print(fs["id"])

In [2]:
def make_prompt(story):
    words = story.split(" ")
    cut = np.random.randint(len(words)//3, len(words)//2)
    # print(len(words), cut)
    cut_story = " ".join(words[:cut])
    # print(cut_story)
    return cut_story

In [4]:
fs_incl_prompt = []
with open(f"full_stories_{lang}.json", "r") as f:
    full_stories = json.load(f)
for fs in tqdm(full_stories):
    prompt = make_prompt(fs["story"])
    fs["prompt"] = prompt
    fs_incl_prompt.append(fs)
with open(f"full_stories_{lang}.json", "w") as f:
    json.dump(fs_incl_prompt, f, indent="    ", ensure_ascii=False)

100%|██████████| 100/100 [00:00<00:00, 10400.73it/s]
